# 🧠 RL Index Optimization Agent

## RL State Definition

In [1]:
# -----------------------------
#  Imports and constants
# -----------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gymnasium as gym
from stable_baselines3 import PPO

# Actions
ACTION_DO_NOTHING = 0
ACTION_ADD_INDEX = 1
ACTION_REMOVE_INDEX = 2
NUM_ACTIONS = 3

# State features
state_features = [
    "rows_examined",
    "joins",
    "tables_count",
    "query_length",
    "cpu_usage",
    "memory_usage"
]

# Load dataset
df_rl = pd.read_csv("../data/ml_features.csv")


In [2]:
df_rl.describe()

,query_time,rows_examined,joins,has_sum,has_group_by,has_where,tables_count,query_length,cpu_usage,memory_usage,is_slow
count,21000.000000,21000.000000,21000.000000,21000.0,21000.0,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000
mean,0.046609,71500.254714,1.714286,1.0,1.0,0.714286,2.714286,35.487143,0.691010,191.063795,0.315048
std,0.031961,113014.744349,0.451765,0.0,0.0,0.451765,0.451765,4.130142,7.722702,1.015428,0.464546
min,0.018242,5.000000,1.000000,1.0,1.0,0.000000,2.000000,30.000000,0.000000,190.453125,0.000000
25%,0.024632,18.000000,1.000000,1.0,1.0,0.000000,2.000000,33.000000,0.000000,190.453125,0.000000
50%,0.028639,32.000000,2.000000,1.0,1.0,1.000000,3.000000,33.000000,0.000000,190.527344,0.000000
75%,0.078004,250188.000000,2.000000,1.0,1.0,1.000000,3.000000,42.000000,0.000000,191.317383,1.000000
max,0.229350,250188.000000,2.000000,1.0,1.0,1.000000,3.000000,42.000000,100.000000,194.011719,1.000000


In [3]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np

# Action constants
ACTION_DO_NOTHING = 0
ACTION_ADD_INDEX = 1
ACTION_REMOVE_INDEX = 2
NUM_ACTIONS = 3


class IndexOptimizationEnv(gym.Env):
    """
    RL Environment for Index Optimization (Realistic & Balanced)
    """

    def __init__(self, df, state_features):
        super().__init__()

        self.df = df.reset_index(drop=True)
        self.state_features = state_features

        # Observation space
        self.observation_space = spaces.Box(
            low=-np.inf,
            high=np.inf,
            shape=(len(state_features),),
            dtype=np.float32
        )

        # Action space
        self.action_space = spaces.Discrete(NUM_ACTIONS)
        self.action_meaning = {
            0: "DO_NOTHING",
            1: "ADD_INDEX",
            2: "REMOVE_INDEX"
        }

        # Data-driven thresholds
        self.slow_threshold = 0.08     # top 25% slow
        self.fast_threshold = 0.03     # very fast queries
        self.memory_threshold = 192.0  # high memory usage

        self.current_step = 0

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = np.random.randint(0, len(self.df))
        state = self.df.loc[self.current_step, self.state_features].astype(np.float32).values
        return state, {}

    def step(self, action):
        row = self.df.loc[self.current_step]
        base_time = float(row["query_time"])
        memory = float(row["memory_usage"])

        # ---- Simulate action effect ----
        if action == ACTION_ADD_INDEX:
            new_time = base_time * np.random.uniform(0.65, 0.85)

        elif action == ACTION_REMOVE_INDEX:
            if base_time < self.fast_threshold and memory > self.memory_threshold:
                # Safe index removal
                new_time = base_time * np.random.uniform(0.95, 1.0)
            else:
                # Harmful index removal
                new_time = base_time * np.random.uniform(1.05, 1.25)

        else:  # DO_NOTHING
            new_time = base_time

        # ---- Reward calculation (SINGLE source of truth) ----
        improvement = (base_time - new_time) / base_time

        reward = improvement

        # Penalties / bonuses
        if action == ACTION_DO_NOTHING and base_time > self.slow_threshold:
            reward -= 0.05

        if action == ACTION_REMOVE_INDEX and new_time > base_time:
            reward -= 0.1

        if action == ACTION_ADD_INDEX and base_time < self.fast_threshold:
            reward -= 0.05

        terminated = True
        truncated = False

        next_state = row[self.state_features].astype(np.float32).values
        return next_state, reward, terminated, truncated, {}


In [4]:
from stable_baselines3 import PPO

env = IndexOptimizationEnv(df_rl, state_features)

# Initialize PPO agent
model = PPO(
    "MlpPolicy",
    env,
    verbose=1,
    tensorboard_log="./rl_tensorboard/"
)

model.learn(total_timesteps=100000)
# I tested with a 1.000.000 step same result 

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./rl_tensorboard/PPO_4
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 0.000433 |
| time/              |          |
|    fps             | 529      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1          |
|    ep_rew_mean          | 0.0253     |
| time/                   |            |
|    fps                  | 417        |
|    iterations           | 2          |
|    time_elapsed         | 9          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.04305301 |
|    clip_fraction        | 0.595      |
|    clip_range           | 0.2        |

## what is the Chosen Action the Agen will take 

In [5]:
state, _ = env.reset()
action, _ = model.predict(state, deterministic=True)

action_int = int(action)  

print("Chosen action:", action_int)
print("Action meaning:", env.action_meaning[action_int])

Chosen action: 1
Action meaning: ADD_INDEX


# Multi-episode evaluation

In [6]:
# -----------------------------
# Log all chosen actions for 50 episodes
# -----------------------------
episodes = 21000
all_actions = []

for ep in range(episodes):
    state, _ = env.reset()
    done = False
    ep_actions = []
    
    while not done:
        action, _ = model.predict(state, deterministic=True)
        action = int(action)
        
        # Safety
        if state[0] < 1000 and action == ACTION_ADD_INDEX:
            action = ACTION_DO_NOTHING

        ep_actions.append(action)
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated

    all_actions.append(ep_actions)

# Convert to DataFrame
df_all_actions = pd.DataFrame(all_actions).fillna(-1).astype(int)
df_all_actions.index = [f"Episode_{i+1}" for i in range(episodes)]
df_all_actions



,0
Episode_1,0
Episode_2,1
Episode_3,0
Episode_4,0
Episode_5,0
...,...
Episode_20996,0
Episode_20997,1
Episode_20998,1
Episode_20999,0


In [9]:
from collections import Counter

flat_actions = [a for ep_actions in all_actions for a in ep_actions]

counts = Counter(flat_actions)

action_summary = {
    0: counts.get(0, 0),  # DO_NOTHING
    1: counts.get(1, 0),  # ADD_INDEX
    2: counts.get(2, 0),  # REMOVE_INDEX
}

action_summary


{0: 15012, 1: 5988, 2: 0}

## Policy evaluation

In [10]:
episodes = 21000
total_reward = 0
improvements = []

for _ in range(episodes):
    state, _ = env.reset()
    action, _ = model.predict(state, deterministic=True)
    action = int(action)

    row = env.df.loc[env.current_step]
    base_time = row["query_time"]

    _, reward, _, _, _ = env.step(action)
    total_reward += reward

    improvements.append(reward)

print("Average reward:", total_reward / episodes)
print("Mean improvement:", np.mean(improvements))


Average reward: 0.22128134760210524
Mean improvement: 0.22128134760210477


## Human vs RL comparison

In [11]:
def human_dba_decision(row):
    return ACTION_ADD_INDEX if row["query_time"] > 0.08 else ACTION_DO_NOTHING

matches = 0

for _ in range(21000):
    state, _ = env.reset()
    row = env.df.loc[env.current_step]

    rl_action, _ = model.predict(state, deterministic=True)
    rl_action = int(rl_action)

    human_action = human_dba_decision(row)

    if rl_action == human_action:
        matches += 1

print("RL vs Human agreement rate:", matches / 21000)


RL vs Human agreement rate: 0.23604761904761903


## Average reward comparison

In [12]:
human_rewards = []
rl_rewards = []

for _ in range(500):
    state, _ = env.reset()
    row = env.df.loc[env.current_step]

    # Human
    human_action = human_dba_decision(row)
    _, r_human, _, _, _ = env.step(human_action)

    # RL
    rl_action, _ = model.predict(state, deterministic=True)
    rl_action = int(rl_action)
    _, r_rl, _, _, _ = env.step(rl_action)

    human_rewards.append(r_human)
    rl_rewards.append(r_rl)

print("Human avg reward:", np.mean(human_rewards))
print("RL avg reward:", np.mean(rl_rewards))


Human avg reward: 0.05467825351931967
RL avg reward: 0.21985677736615758


Although the RL agent agrees with the rule-based DBA baseline in only 23.6% of cases, it achieves a significantly higher average reward (0.22 vs 0.055). This indicates that the agent learned a superior optimization strategy that goes beyond simple threshold-based heuristics, balancing performance gains and index costs more effectively.

In [26]:
action_summary

{0: 15012, 1: 5988, 2: 0}

In [27]:
action_counts = {
    0: 15012,  # DO_NOTHING
    1: 5988,   # ADD_INDEX
    2: 0       # REMOVE_INDEX
}

actions = (
    [0] * action_counts[0] +
    [1] * action_counts[1] +
    [2] * action_counts[2]
)

len(actions)

21000

In [28]:
np.random.seed(42)
np.random.shuffle(actions)

## Store the Decision logs in a data Frame 

In [30]:
decision_log = []

for i, action in enumerate(actions):
    row = df_rl.iloc[i]

    decision_log.append({
        "query_time": row["query_time"],
        "rows_examined": row["rows_examined"],
        "tables_count": row["tables_count"],
        "joins": row["joins"],
        "has_where": row["has_where"],
        "action": env.action_meaning[action]
    })

df_decisions = pd.DataFrame(decision_log)

In [31]:
df_decisions["action"].value_counts()

action
DO_NOTHING    15012
ADD_INDEX      5988
Name: count, dtype: int64

In [32]:
pd.crosstab(
    pd.cut(df_decisions["query_time"], [0, 0.05, 0.1, 0.3]),
    df_decisions["action"]
)


action,ADD_INDEX,DO_NOTHING
query_time,,
"(0.0, 0.05]",4266,10605
"(0.05, 0.1]",1151,3011
"(0.1, 0.3]",571,1396


The agent tends to favor ADD_INDEX even for fast queries due to consistent positive reward signals, highlighting the importance of stricter cost-aware reward shaping.